In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from process_video_ben import process_video
import pandas as pd
import numpy as np
import os

2025-06-12 11:00:14.476991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749693614.660421   14359 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749693614.745496   14359 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-12 11:00:15.226902: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
metadata = pd.read_csv("/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/data/preprocessed/landmarks_metadata_v4.csv")
metadata.head()

,filename,filename_accented,label,label_accented,accented,data_source,original_fps,original_frame_count,original_duration_sec,start_frame,end_frame,processed_frame_count,processed_duration_sec,preprocess_version,face_width,shoulders_width,face_midpoint_to_shoulders_height,left_hand_interpolated_none_frames,right_hand_interpolated_none_frames
0,ajudar_ne_1.mp4,ajudar_ne_1.mp4,ajudar,ajudar,False,ne,12.00000,44,3.666667,7,36,30,2.500000,v4,0.147576,0.333421,0.292776,7,6
1,ajudar_sb_2.mp4,ajudar_sb_2.mp4,ajudar,ajudar,False,sb,29.97003,79,2.635967,6,66,61,2.035367,v4,0.086359,0.206200,0.216821,0,0
2,ajudar_uf_3.mp4,ajudar_uf_3.mp4,ajudar,ajudar,False,uf,29.97003,115,3.837167,24,89,66,2.202200,v4,0.090604,0.205890,0.222293,0,0
3,ajudar_vl_4.mp4,ajudar_vl_4.mp4,ajudar,ajudar,False,vl,29.97003,141,4.704700,11,126,116,3.870533,v4,0.100236,0.210215,0.240169,25,9
4,ajudar_vl_5.mp4,ajudar_vl_5.mp4,ajudar,ajudar,False,vl,29.97000,108,3.603604,0,106,107,3.570237,v4,0.114839,0.243479,0.246769,13,12


In [5]:
metadata["filename_accented"][42:]

42     cabeça_ne_1.mp4
43     cabeça_sb_2.mp4
44     cabeça_uf_3.mp4
45     cabeça_vl_4.mp4
46     cabeça_vl_5.mp4
            ...       
145    vagina_sb_2.mp4
146    vagina_uf_3.mp4
147    vagina_vl_4.mp4
148    vagina_vl_5.mp4
149    vagina_vl_6.mp4
Name: filename_accented, Length: 108, dtype: object

In [ ]:
output_dict = {}

for fn in metadata["filename_accented"]:
    prediction, probs_str, label_encoding = process_video(fn)
    output_dict[fn] = {
        "prediction": prediction,
        "probs_str": probs_str,
        "label_encoding": label_encoding
    }

Check Model Predictions

In [35]:
model_metadata = pd.read_csv("/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/modelling/metadata/landmarks_metadata_v4_training.csv")
train_metadata = model_metadata[model_metadata["dataset_split"] == "train"]
test_metadata = model_metadata[model_metadata["dataset_split"] == "test"]

In [36]:
output_folder = "/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/webapp/backend/data/output/"
train_eval = []
for fn in train_metadata["filename_accented"]:
    fn_txt = fn.replace(".mp4", ".txt")
    fn_txt_fp = os.path.join(output_folder, fn_txt)
    with open(fn_txt_fp, "r") as f:
        lines = f.readlines()
    pred = lines[1]
    pred_class = int(pred.split(":")[0].split(" ")[1])
    pred_label = pred.split(":")[1].strip()
    max_prob = np.max([float(num) for num in [line.split(' ')[-1] for line in lines[2].split(",")][:-1]])
    print(fn, pred_class, pred_label, max_prob)
    if fn.split('_')[0] == pred_label:
        train_eval.append(1)
    else:
        train_eval.append(0)

ajudar_ne_1.mp4 15 ajudar 0.999
ajudar_sb_2.mp4 15 ajudar 1.0
ajudar_uf_3.mp4 15 ajudar 0.999
ajudar_vl_4.mp4 15 ajudar 0.999
ajudar_vl_6.mp4 15 ajudar 0.999
animal_ne_1.mp4 21 animal 1.0
animal_sb_2.mp4 21 animal 0.999
animal_uf_3.mp4 21 animal 1.0
animal_vl_5.mp4 21 animal 0.999
animal_vl_6.mp4 21 animal 0.999
aniversário_ne_1.mp4 3 aniversário 1.0
aniversário_sb_2.mp4 3 aniversário 1.0
aniversário_vl_4.mp4 3 aniversário 1.0
aniversário_vl_5.mp4 3 aniversário 1.0
aniversário_vl_6.mp4 3 aniversário 1.0
ano_ne_1.mp4 6 ano 1.0
ano_uf_3.mp4 6 ano 1.0
ano_vl_4.mp4 6 ano 1.0
ano_vl_5.mp4 6 ano 1.0
ano_vl_6.mp4 6 ano 1.0
banana_sb_2.mp4 9 banana 1.0
banana_uf_3.mp4 9 banana 1.0
banana_vl_4.mp4 9 banana 1.0
banana_vl_5.mp4 9 banana 1.0
banana_vl_6.mp4 9 banana 0.999
banheiro_ne_1.mp4 18 banheiro 0.999
banheiro_sb_2.mp4 18 banheiro 1.0
banheiro_uf_3.mp4 18 banheiro 1.0
banheiro_vl_4.mp4 18 banheiro 1.0
banheiro_vl_5.mp4 18 banheiro 1.0
bebê_ne_1.mp4 2 bebê 1.0
bebê_sb_2.mp4 2 bebê 1.0
bebê_uf

In [33]:
output_folder = "/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/webapp/backend/data/output/"
test_eval = []
for fn in test_metadata["filename_accented"]:
    fn_txt = fn.replace(".mp4", ".txt")
    fn_txt_fp = os.path.join(output_folder, fn_txt)
    with open(fn_txt_fp, "r") as f:
        lines = f.readlines()
    pred = lines[1]
    pred_class = int(pred.split(":")[0].split(" ")[1])
    pred_label = pred.split(":")[1].strip()
    max_prob = np.max([float(num) for num in [line.split(' ')[-1] for line in lines[2].split(",")][:-1]])
    print(fn, pred_class, pred_label, max_prob)
    if fn.split('_')[0] == pred_label:
        test_eval.append(1)
    else:
        test_eval.append(0)

ajudar_vl_5.mp4 6 ano 0.871
animal_vl_4.mp4 21 animal 0.933
aniversário_uf_3.mp4 3 aniversário 0.999
ano_sb_2.mp4 6 ano 1.0
banana_ne_1.mp4 9 banana 0.987
banheiro_vl_6.mp4 18 banheiro 0.989
bebê_vl_5.mp4 2 bebê 0.998
cabeça_vl_4.mp4 1 cabeça 0.997
café_uf_3.mp4 16 carne 0.523
carne_sb_2.mp4 15 ajudar 0.931
casa_ne_1.mp4 19 família 0.998
cebola_vl_6.mp4 17 sorvete 0.574
comer_vl_5.mp4 8 garganta 0.964
cortar_vl_4.mp4 23 filho 0.596
crescer_uf_3.mp4 5 crescer 0.991
família_sb_2.mp4 19 família 0.999
filho_ne_1.mp4 23 filho 0.948
garganta_vl_6.mp4 0 homem 0.999
homem_vl_5.mp4 8 garganta 0.942
jovem_vl_4.mp4 4 jovem 0.999
ouvir_uf_3.mp4 10 ouvir 0.999
pai_sb_2.mp4 22 pai 0.8
sopa_ne_1.mp4 21 animal 0.447
sorvete_vl_6.mp4 17 sorvete 1.0
vagina_vl_5.mp4 12 vagina 1.0


In [38]:
print(f'Train Accuracy: {np.mean(train_eval)*100:.2f}% (n={len(train_eval)})')
print(f'Test Accuracy: {np.mean(test_eval)*100:.2f}% (n={len(test_eval)})')

Train Accuracy: 100.00% (n=125)
Test Accuracy: 60.00% (n=25)


Check Backend Preprocessing Matches Development/Training Preprocessing

In [15]:
def landmark_to_array(landmarks):
    lm_arrays = {
        "left_hand_landmarks":[], 
        "right_hand_landmarks":[], 
        "pose_landmarks":[],
        "face_landmarks":[]
    }
    for frame in landmarks:
        for key in lm_arrays.keys():
            lm_array = [[lm.x, lm.y, lm.z] for lm in frame[f"{key}"].landmark]
            lm_arrays[key].append(lm_array)

    return lm_arrays

In [16]:
np_check_dict = {}

orig_npy_folder = "/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/data/preprocessed/landmarks/v4"
new_npy_folder = "/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/webapp/backend/data/preprocessed/landmarks/v0"

for i, row in metadata.iterrows():
    fn = row["filename"]
    fn_a = row["filename_accented"]
    fn = fn.replace(".mp4", ".npy")
    fn_a = fn_a.replace(".mp4", ".npy")
    orig_npy_fp = os.path.join(orig_npy_folder, fn)
    new_npy_fp = os.path.join(new_npy_folder, fn_a)
    orig_npy = np.load(orig_npy_fp, allow_pickle=True)
    new_npy = np.load(new_npy_fp, allow_pickle=True)
    orig_lm_arrays = landmark_to_array(orig_npy)
    new_lm_arrays = landmark_to_array(new_npy)
    check_dict = {
    "left_hand_landmarks":True, 
    "right_hand_landmarks":True, 
    "pose_landmarks":True,
    "face_landmarks":True
    }
    for key in check_dict.keys():
        if not np.array_equal(orig_lm_arrays[key], new_lm_arrays[key]):
            check_dict[key] = False
    np_check_dict[fn] = check_dict

In [22]:
check_df = pd.DataFrame(np_check_dict).T

In [23]:
check_df[check_df["left_hand_landmarks"] == False]

,left_hand_landmarks,right_hand_landmarks,pose_landmarks,face_landmarks


In [24]:
check_df[check_df["right_hand_landmarks"] == False]

,left_hand_landmarks,right_hand_landmarks,pose_landmarks,face_landmarks


In [25]:
check_df[check_df["pose_landmarks"] == False]

,left_hand_landmarks,right_hand_landmarks,pose_landmarks,face_landmarks


In [26]:
check_df[check_df["face_landmarks"] == False]

,left_hand_landmarks,right_hand_landmarks,pose_landmarks,face_landmarks
